In [1]:
import sys
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Spark Session") \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS gold")

DataFrame[]

In [37]:
df = spark.sql("""
SELECT
Data,
Data_Pgto,
Descricao,
Tipo,
'Extrato' AS Fonte,
Valor
FROM silver.nubank_extrato        
WHERE Identificador NOT IN
(
    SELECT Identificador FROM silver.nubank_extrato
    WHERE Descricao = 'Pagamento de fatura' AND Data >= '2024-01-01'
)
UNION ALL
SELECT 
Data,
Data_Pgto,
Descricao,
Tipo,
'Fatura' AS Fonte,
Valor
FROM silver.nubank_fatura
Where Descricao != 'Pagamento recebido'
""")
df.createOrReplaceTempView('movimentacao')


In [38]:
df.coalesce(1).write.mode("overwrite").format("parquet").saveAsTable(f"gold.nubank_movimentacao")

In [39]:
spark.sql("""
SELECT
sum(valor)
FROM movimentacao
""").show()

+-----------------+
|       sum(valor)|
+-----------------+
|1930.389999999912|
+-----------------+

